In [1]:
import numpy as np
import json
import os

In [2]:
# read multiplication table file
# Windows infile
#infile = [np.asarray(json.loads(line)) for line in open(os.path.abspath(os.pardir) + '\z2_s4_001.json', 'r')]
# OS infile
infile = [np.asarray(json.loads(line)) for line in open(os.path.abspath(os.pardir) + '/z2_s4_001.json', 'r')]
mixer_index = infile[0]
M = infile[1] #multipliers
T = infile[2] #multiplication table

assert T.shape[0] == T.shape[1] == T.shape[2]
len_T = T.shape[0]
print("Number of conjugacy classes: {}" .format(len_T))


assert M.shape[1] == len_T
len_M = M.shape[0]
print("Number of multipliers: {}" .format(len_M))

#print("Multiplier Indices: {}" .format(M))

Number of conjugacy classes: 33
Number of multipliers: 9


In [3]:
# Convert multplication table from 3D array of integers to 3D list of bin
def array_to_list(X):
    return X.tolist()

M1 = array_to_list(M)
T1 = array_to_list(T)

In [9]:
nbytes = 2
PAD = '0'

def pad_string(m, len_T, PAD):
    pad_length = (len_T - len(m) % len_T)
    pad = PAD*pad_length
    return m + pad, pad_length

def string_to_byte_sequence(m, nbytes):
    #convert string to binary sequence
    #print(bin_version)
    return ''.join(format(ord(i), '08b') for i in m)

def byte_sequence_chunk(m, nbytes):
    #seperate byte sequence into list of 'chunks' of length 'bits_per_coefficient'
    bits_per_coefficient = nbytes * 2
    res = []
    for start in range(0, len(m), bits_per_coefficient):
        res.append(m[start:start + bits_per_coefficient])
    return res

def byte_sequence_to_binary_sequence(m):
    #interpret each byte chunk as binary integer 
    return [int(e,2) for e in m]

def binary_block_sequence(m, len_T):
    #seperate chunks into blocks of length 'len_T'
    return [m[i:i + len_T] for i in range(0, len(m), len_T)]

def binary_block_sequence_to_byte_sequence(m, nbytes):
    res = ''
    chunk_size = nbytes*2
    for i in range(len(m)):
        for j in range(len(m[i])):
            byte = str(bin(m[i][j])[2:].zfill(chunk_size))
            res += byte
    return res

def byte_to_pad_string(m):
    return ''.join(chr(int(m[i*8:i*8+8],2)) for i in range(len(m)//8))

def unpad_string(m, pad_length):
    return m[:len(m)-pad_length]
    

In [6]:
def string_to_bring(m, len_T, nbytes = 2, PAD = '0'):
    m1, pad_length = pad_string(m, len_T, PAD)
    m2 = string_to_byte_sequence(m1, nbytes)
    m3 = byte_sequence_chunk(m2, nbytes)
    m4 = byte_sequence_to_binary_sequence(m3)
    m5 = binary_block_sequence(m4, len_T)
    return m5, pad_length

def bring_to_string(m, len_T, pad_length, nbytes = 2, PAD = '0'):
    m1 = binary_block_sequence_to_byte_sequence(m, nbytes)
    m2 = byte_to_pad_string(m1)
    m3 = unpad_string(m2, pad_length)
    return m3

In [39]:
# Demonstration
m = 'abcdefg'
print("String: %s; Length: %s" % (m, len(m)))
m1, pad_length = pad_string(m, len_T, PAD)
print("Padded String: %s; Length: %s" % (m1, len(m1)))
print("Pad Length: %s" % (pad_length))
m2 = string_to_byte_sequence(m1, nbytes)
print("Byte Sequence: %s; Length: %s" % (m2, len(m2)))
m3 = byte_sequence_chunk(m2, nbytes)
print("Byte Sequence Chunks: %s" % (m3))
m4 = byte_sequence_to_binary_sequence(m3)
print("Binary Sequence: %s" % (m4))
m5 = binary_block_sequence(m4, len_T)
print("Binary Sequence Blocks: %s" % (m5))
m6 = binary_block_sequence_to_byte_sequence(m5, nbytes)
print("Byte Sequence: %s; Length: %s" % (m6, len(m6)))
m7 = byte_to_pad_string(m6)
print("Reconstructed Padded String: %s" % (m7))
m8 = unpad_string(m7, pad_length)
print("Reconstructed String: %s" % (m8))

String: abcdefg; Length: 7
Padded String: abcdefg00000000000000000000000000; Length: 33
Pad Length: 26
Byte Sequence: 011000010110001001100011011001000110010101100110011001110011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000001100000011000000110000; Length: 264
Byte Sequence Chunks: ['0110', '0001', '0110', '0010', '0110', '0011', '0110', '0100', '0110', '0101', '0110', '0110', '0110', '0111', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000', '0011', '0000']
Binary Sequence: [6, 1, 6, 2, 6, 3, 6, 4, 6, 5, 6, 6,

In [7]:
def brmult_list(T, L, sel = "none"):
    if sel == "none":
        pass
    else:
        L = L[sel,:]
        
    for l in L:
        if all(l == L[0]): # initialize product
            chi = l
        else:
            temp = np.zeros(len(l))
            for i in range(len(l)):
                for j in range(len(l)):
                    temp+= chi[i]*l[j]*T[i][j]
            chi = temp
            #print("-------------------------------------------------------------")
            #print(chi) 
    return chi

In [13]:
msg = "abcdefghijklmnopqrs"
msg_rep, pad_length = string_to_bring(msg, len_T)
print(msg_rep)

[[6, 1, 6, 2, 6, 3, 6, 4, 6, 5, 6, 6, 6, 7, 6, 8, 6, 9, 6, 10, 6, 11, 6, 12, 6, 13, 6, 14, 6, 15, 7, 0, 7], [1, 7, 2, 7, 3, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0, 3, 0]]


In [23]:
multiplier = brmult_list(T,M)
encryption = brmult_list(T,[msg_rep,multiplier])
#T1[0][0]

TypeError: 'bool' object is not iterable

In [98]:
#encryption = [brmult_list(T,[e,multiplier]) for e in msg_utf]
#encryption = brmult_list(T,[msg_utf,multiplier])
print(encryption)

[array([ 2.222e+03, -1.333e+03,  3.654e+03, -1.010e+03, -2.442e+03,
       -2.343e+03, -2.210e+02, -2.432e+03, -2.210e+02, -1.533e+03,
       -1.452e+03, -1.541e+03, -2.532e+03,  2.553e+03, -2.443e+03,
        1.000e+03,  1.000e+00, -8.900e+02,  1.000e+00, -8.990e+02,
        1.000e+00,  1.232e+03,  3.310e+02,  1.321e+03,  1.000e+00,
        1.322e+03,  1.000e+00, -1.332e+03, -3.320e+02, -1.222e+03,
       -2.220e+02, -1.110e+02,  1.110e+02]), array([  15.,  -17.,   30.,   -4.,  -19.,  -19.,  -13.,  -18.,  -13.,
        -18.,  -18.,  -18.,  -18.,   28.,  -18.,    2.,    9.,    9.,
          9.,    9.,    9.,   14.,   14.,   14.,    9.,   14.,    9.,
        -24.,  -23., -122.,  -21., -122.,   11.])]


In [99]:
decryption = [brmult_list(T,[e,multiplier]).astype(int) for e in encryption]
print(decryption)


[array([ 110,    1,  110,   10,  110,   11,  110,  100,  110,  101,  110,
        110,  110,  111,  110, 1000,  110, 1001,  110, 1010,  110, 1011,
        110, 1100,  110, 1101,  110, 1110,  110, 1111,  111,    0,  111]), array([  2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   1, 111,  10, 111,  11])]


In [93]:
def to_bytestr(m, nbytes =2):
    return ''.join(chr(m[i*8:i*8+8],2 for i in range(len(m)//8))

In [94]:
msg_rec = to_bytestr(decryption[0])

TypeError: chr() takes exactly one argument (2 given)